In [4]:
!pip install taskingai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [5]:
import time
import os
import taskingai


In [6]:
# Load TaskingAI API Key from environment variable
from google.colab import userdata
# Set the environment variable to be the value stored in user secrets.
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["TASKINGAI_API_KEY"] =  userdata.get('TASKINGAI_API_KEY')

In [7]:
# Taskingai initianlization
taskingai.init(api_key=os.environ["TASKINGAI_API_KEY"])

# TaskingAI: Chat with Assistant Example

In this example, we will first create an assistant who knows the meaning of various numbers and will explain it in certain language.
Then we will start a chat with the assistant.

##  Create Assistant

In [8]:
from taskingai.tool import Action, ActionAuthentication, ActionAuthenticationType
from typing import List

# create an assistant action
NUMBERS_API_SCHEMA = {
        "openapi": "3.0.0",
        "info": {
            "title": "Numbers API",
            "version": "1.0.0",
            "description": "API for fetching interesting number facts"
        },
        "servers": [
            {
            "url": "http://numbersapi.com"
            }
        ],
        "paths": {
            "/{number}": {
            "get": {
                "description": "Get a fact about a number",
                "operationId": "getNumberFact",
                "parameters": [
                {
                    "name": "number",
                    "in": "path",
                    "required": True,
                    "description": "The number to get the fact for",
                    "schema": {
                    "type": "integer"
                    }
                }
                ],
                "responses": {
                "200": {
                    "description": "A fact about the number",
                    "content": {
                    "text/plain": {
                        "schema": {
                        "type": "string"
                        }
                    }
                    }
                }
                }
            }
        }
    }
}
actions: List[Action] = taskingai.tool.bulk_create_actions(
    openapi_schema=NUMBERS_API_SCHEMA,
    authentication=ActionAuthentication(
        type=ActionAuthenticationType.NONE,
    )
)
action = actions[0]
print(f"created action: {action}\n")

created action: object='Action' action_id='bFBdZYGFIKT60qU6Gkqxfz41' name='get_number_fact' operation_id='getNumberFact' description='Get a fact about a number' url='http://numbersapi.com/{number}' method=<ActionMethod.GET: 'GET'> body_type=<ActionBodyType.NONE: 'NONE'> openapi_schema={'info': {'title': 'Numbers API', 'version': '1.0.0', 'description': 'API for fetching interesting number facts'}, 'paths': {'/{number}': {'get': {'responses': {'200': {'content': {'text/plain': {'schema': {'type': 'string'}}}, 'description': 'A fact about the number'}}, 'parameters': [{'in': 'path', 'name': 'number', 'schema': {'type': 'integer'}, 'required': True, 'description': 'The number to get the fact for'}], 'description': 'Get a fact about a number', 'operationId': 'getNumberFact'}}}, 'openapi': '3.0.0', 'servers': [{'url': 'http://numbersapi.com'}], 'security': [], 'components': {}} authentication=ActionAuthentication(type=<ActionAuthenticationType.NONE: 'none'>, secret=None, content=None) updat

In [10]:
from taskingai.assistant import Assistant, Chat, ToolRef, ToolType
from taskingai.assistant.memory import AssistantMessageWindowMemory

# choose an available chat_completion model from your project
model_id = "TpQuiRBb" #gpt4

assistant: Assistant = taskingai.assistant.create_assistant(
    model_id=model_id,
    name="My Assistant",
    description="A assistant who knows the meaning of various numbers.",
    memory=AssistantMessageWindowMemory(
        max_messages=20,
        max_tokens=1000
    ),
    system_prompt_template=[
        "You know the meaning of various numbers.",
        "No matter what the user's language is, you will use the {{langugae}} to explain."
    ],
    tools=[
        ToolRef(
            type=ToolType.ACTION,
            id=action.action_id,
        )
    ],
    retrievals=[],
    metadata={"k": "v"},
)
print(f"created assistant: {assistant}\n")

created assistant: object='Assistant' assistant_id='X5lMOCa2U7PHGOCL4b7Olbli' model_id='TpQuiRBb' name='My Assistant' description='A assistant who knows the meaning of various numbers.' system_prompt_template=['You know the meaning of various numbers.', "No matter what the user's language is, you will use the {{langugae}} to explain."] memory=AssistantMemory(type=<AssistantMemoryType.MESSAGE_WINDOW: 'message_window'>, max_messages=20, max_tokens=1000) tools=[ToolRef(type=<ToolType.ACTION: 'action'>, id='bFBdZYGFIKT60qU6Gkqxfz41')] retrievals=[] retrieval_configs=RetrievalConfig(top_k=3, max_tokens=None, score_threshold=None, method=<RetrievalMethod.USER_MESSAGE: 'user_message'>, function_description=None) metadata={'k': 'v'} updated_timestamp=1716826741139 created_timestamp=1716826741139



## Start a Chat

In [11]:
chat: Chat = taskingai.assistant.create_chat(
    assistant_id=assistant.assistant_id,
)
print(f"created chat: {chat.chat_id}\n")

created chat: SdELaXlQ76dAzh5s3NxLKJmH



In [12]:
from taskingai.assistant import Message, MessageChunk
user_input = input("User Input: ")
while user_input.strip() and user_input != "q":
    # create user message
    taskingai.assistant.create_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        text=user_input,
    )
    print(f"User: {user_input}")

    # generate assistant response
    assistant_message: Message = taskingai.assistant.generate_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        system_prompt_variables={
            "language": "English"
        }
    )
    print(f"Assistant: {assistant_message.content.text}")
    time.sleep(2)
    # quit by input 'q
    user_input = input("User: ")

User Input: who is Oppenheimer
User: who is Oppenheimer
Assistant: J. Robert Oppenheimer was an American theoretical physicist, most famously known as the "father of the atomic bomb". He was a key contributor to the Manhattan Project, the World War II project that developed the first nuclear weapons. Post-war, Oppenheimer became a advocate for controlling nuclear power to avoid nuclear conflict and arms race.
User: q


In [13]:
user_input = input("User Input: ")
while user_input.strip() and user_input != "q":
    # create user message
    user_message = taskingai.assistant.create_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        text=user_input,
    )
    print(f"User: {user_input} ({user_message.message_id})")

    # generate assistant response
    assistant_message_response = taskingai.assistant.generate_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        system_prompt_variables={
            "language": "English"
        },
        stream=True,
    )

    print(f"Assistant:", end=" ", flush=True)
    for item in assistant_message_response:
        if isinstance(item, MessageChunk):
            print(item.delta, end="", flush=True)
        elif isinstance(item, Message):
            print(f" ({item.message_id})")

    time.sleep(2)
    # quit by input 'q
    user_input = input("User: ")

User Input: what did he do
User: what did he do (Mah1tewOwx2TcT8NseKPTPSk)
Assistant: J. Robert Oppenheimer played a significant role in the development of atomic weapons. Here are his main contributions:

1. Manhattan Project: A governmental research project during World War II, the Manhattan Project was directed towards making atomic weapons. Oppenheimer was appointed as the lead scientific director. Despite numerous challenges, he created a successful team which developed and detonated the first atomic bomb in 1945 in New Mexico.

2. Creation of modern theoretical physics: Oppenheimer heavily contributed to the field of theoretical physics. He is known to have predicted the existence of black holes and neutron stars before they were discovered.

3. Advocating for nuclear control: After the devastative use of atomic bombs in Hiroshima and Nagasaki, Oppenheimer became a strong advocate for controlling nuclear power. He worked diligently to make sure nuclear power was not used for dest

In [14]:
# list messages
messages = taskingai.assistant.list_messages(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
    order="asc"
)
for message in messages:
    print(f"{message.role}: {message.content.text}")

user: who is Oppenheimer
assistant: J. Robert Oppenheimer was an American theoretical physicist, most famously known as the "father of the atomic bomb". He was a key contributor to the Manhattan Project, the World War II project that developed the first nuclear weapons. Post-war, Oppenheimer became a advocate for controlling nuclear power to avoid nuclear conflict and arms race.
user: what did he do
assistant: J. Robert Oppenheimer played a significant role in the development of atomic weapons. Here are his main contributions:

1. Manhattan Project: A governmental research project during World War II, the Manhattan Project was directed towards making atomic weapons. Oppenheimer was appointed as the lead scientific director. Despite numerous challenges, he created a successful team which developed and detonated the first atomic bomb in 1945 in New Mexico.

2. Creation of modern theoretical physics: Oppenheimer heavily contributed to the field of theoretical physics. He is known to have 

In [15]:
# delete assistant
taskingai.assistant.delete_assistant(
    assistant_id=assistant.assistant_id,
)